In [1]:
# Mounting drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
! unzip gdrive/My\ Drive/fer2013.zip

Archive:  gdrive/My Drive/fer2013.zip
  inflating: fer2013.csv             


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
num_classes = 7 # angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [0]:
# with open("fer2013.csv") as f:
#   content = f.readlines()
 
# lines = np.array(content)
 
# num_of_instances = lines.size
# print("number of instances: ",num_of_instances)

In [0]:
# for i in range(1, num_of_instances):
  
#   try:
#     emotion, img, usage = lines[i].split(",")
    
#     val = img.split(" ")
#     pixels = np.array(val, 'float32')
    
#     if 'Training' in usage:
#       y_train.append(emotion)
#       x_train.append(pixels)
      
#     elif 'PublicTest' in usage:
#       y_test.append(emotion)
#       x_test.append(pixels)
      
#   except:
#     print("", end="")

In [4]:
import pandas as pd
data = pd.read_csv('fer2013.csv', header=0)
data.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

In [0]:
grouped = data.groupby('Usage')

train = grouped.get_group('Training')
test = grouped.get_group('PublicTest')

In [0]:
df = pd.DataFrame(train)
df.drop(['Usage'], axis=1, inplace=True)
df.to_csv("train.csv", index=False)

# df = pd.DataFrame(y_train)
# df.to_csv("y_train.csv", index=False)

df = pd.DataFrame(test)
df.drop(['Usage'], axis=1, inplace=True)
df.to_csv("test.csv", index=False)

# df = pd.DataFrame(y_test)
# df.to_csv("y_test.csv", index=False)

In [7]:
! pip install torch torchvision

In [0]:
import torch.utils.data as Data
from PIL import Image

In [0]:
class FerDataset(Data.Dataset):
  
  def __init__(self, csv_path, transforms=None, train=False):
    
    self.transform = transforms
    
    #self.split = split
    
    self.train = train
    
    #self.data = pd.read_csv(csv_path)
    
    
    with open(csv_path) as f:
      content = f.readlines()
 
    lines = np.array(content)
 
    num_of_instances = lines.size
    #print("number of instances: ",num_of_instances)
    
    self.train_data, self.train_labels, self.Test_data, self.Test_labels = [], [], [], []
    
    for i in range(1, num_of_instances):
  
      try:
        emotion, img = lines[i].split(",")
        
        val = img.split(" ")
        pixels = np.array(val, 'float32')
      
        if self.train == True:
          self.train_labels.append(emotion)
          self.train_data.append(pixels)
      
        else:
          self.Test_labels.append(emotion)
          self.Test_data.append(pixels)
      
      except:
        print("", end="")
    
    
    
    
    
    
    if self.train == True:
      
#       self.train_data = self.data['pixels']
#       self.train_labels = self.data['emotion']
       self.train_data = np.asarray(self.train_data)
#       print(type(self.train_data))
#       print(type(self.train_data[1]))
       self.train_data = self.train_data.reshape((28709, 48, 48))
     
    
    else: 
      
#       self.Test_data = self.data['pixels']
#       self.Test_labels = self.data['emotion']
       self.Test_data = np.asarray(self.Test_data)
       self.Test_data = self.Test_data.reshape((3589, 48, 48))
      
      
  def __len__(self):
    
    if self.train == True:
      return len(self.train_data)
    
    else:
      return len(self.Test_data)
    
  
  
  def __getitem__(self):
    
    if self.train == True:
      
      img, target = self.train_data, self.train_labels
      
    else:
      
      img, target = self.Test_data, self.Test_labels
      
    img = img[:, :, np.newaxis]
    img = np.concatenate((img, img, img), axis=2)
    img = Image.fromarray(img)
    
    if self.transform is not None:
      
      img = self.transform(img)
        
    return img, target
    
      
      
  
  
  

In [0]:
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(44),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(44),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

In [0]:
trainset = FerDataset('train.csv', transforms=transform_train, train=True)

In [0]:
import torch

In [0]:
trainloader = Data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=1)